### 1. Environment Setup

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "jax"

from IPython.core.magic import register_cell_magic

@register_cell_magic
def backend(line, cell):
    current, required = os.environ.get("KERAS_BACKEND", ""), line.split()[-1]
    if current == required:
        get_ipython().run_cell(cell)
    else:
        print(
            f"This cell requires the {required} backend. To run it, change KERAS_BACKEND to "
            f"\"{required}\" at the top of the notebook, restart the runtime, and rerun the notebook."
        )

### 2. Download and Extract Data

In [ ]:
import kagglehub

kagglehub.login()
download_path = kagglehub.competition_download("dogs-vs-cats")

import zipfile

with zipfile.ZipFile(download_path + "/train.zip", "r") as zip_ref:
    zip_ref.extractall(".")

### 3. Prepare Dataset (Train / Validation / Test)

In [ ]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("dogs_vs_cats_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname, dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

### 4. Load Data

In [ ]:
from keras.utils import image_dataset_from_directory

batch_size = 64
image_size = (180, 180)
train_dataset = image_dataset_from_directory(
    new_base_dir / "train", image_size=image_size, batch_size=batch_size
)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation", image_size=image_size, batch_size=batch_size
)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test", image_size=image_size, batch_size=batch_size
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


I0000 00:00:1756971497.342840  385047 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1756971497.342858  385047 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### 5. Data Augmentation

In [4]:
import keras
from keras import layers
import tensorflow as tf

data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
]

def data_augmentation(images, targets):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images, targets

augmented_train_dataset = train_dataset.map(
    data_augmentation, num_parallel_calls=8
)
augmented_train_dataset = augmented_train_dataset.prefetch(tf.data.AUTOTUNE)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



W0000 00:00:1756971497.528312  385047 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1756971497.528943  385047 service.cc:145] XLA service 0x309656550 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756971497.528952  385047 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1756971497.529825  385047 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1756971497.529836  385047 mps_client.cc:384] XLA backend will use up to 11452841984 bytes on device 0 for SimpleAllocator.


### 6. Load Pretrained Model (Xception)

In [5]:
import keras_hub

conv_base = keras_hub.models.Backbone.from_preset(
    "xception_41_imagenet",
    trainable=False,
)
preprocessor = keras_hub.layers.ImageConverter.from_preset(
    "xception_41_imagenet",
    image_size=(180, 180),
)
conv_base.trainable = False
len(conv_base.trainable_weights)

0

### 7. Build Model (Feature Extraction)

In [6]:
inputs = keras.Input(shape=(180, 180, 3))
x = preprocessor(inputs)
x = conv_base(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

### 8. Train Model (Feature Extraction)

In [7]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction_with_data_augmentation.keras",
        save_best_only=True,
        monitor="val_loss",
    )
]
history = model.fit(
    augmented_train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks,
)

Epoch 1/30


/Users/chan/projects/keras-dl/.venv/lib/python3.11/site-packages/jax/_src/interpreters/mlir.py:1153: UserWarning: Some donated buffers were not usable: ShapedArray(float32[2048,256]), ShapedArray(float32[256]), ShapedArray(float32[256,1]), ShapedArray(float32[1]), ShapedArray(float32[3,3,3,32]), ShapedArray(float32[32]), ShapedArray(float32[32]), ShapedArray(float32[32]), ShapedArray(float32[32]), ShapedArray(float32[3,3,32,64]), ShapedArray(float32[64]), ShapedArray(float32[64]), ShapedArray(float32[64]), ShapedArray(float32[64]), ShapedArray(float32[3,3,64,1]), ShapedArray(float32[1,1,64,128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[3,3,128,1]), ShapedArray(float32[1,1,128,128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[1,1,64,128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 531ms/step - accuracy: 0.8861 - loss: 0.2317

/Users/chan/projects/keras-dl/.venv/lib/python3.11/site-packages/jax/_src/interpreters/mlir.py:1153: UserWarning: Some donated buffers were not usable: ShapedArray(float32[2048,256]), ShapedArray(float32[256]), ShapedArray(float32[256,1]), ShapedArray(float32[1]), ShapedArray(float32[3,3,3,32]), ShapedArray(float32[32]), ShapedArray(float32[32]), ShapedArray(float32[32]), ShapedArray(float32[32]), ShapedArray(float32[3,3,32,64]), ShapedArray(float32[64]), ShapedArray(float32[64]), ShapedArray(float32[64]), ShapedArray(float32[64]), ShapedArray(float32[3,3,64,1]), ShapedArray(float32[1,1,64,128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[3,3,128,1]), ShapedArray(float32[1,1,128,128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[1,1,64,128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]

32/32 ━━━━━━━━━━━━━━━━━━━━ 27s 809ms/step - accuracy: 0.9405 - loss: 0.1686 - val_accuracy: 0.9780 - val_loss: 0.0800
Epoch 2/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 25s 771ms/step - accuracy: 0.9660 - loss: 0.0866 - val_accuracy: 0.9800 - val_loss: 0.0749
Epoch 3/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 26s 821ms/step - accuracy: 0.9525 - loss: 0.1594 - val_accuracy: 0.9810 - val_loss: 0.0598
Epoch 4/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 28s 862ms/step - accuracy: 0.9725 - loss: 0.0748 - val_accuracy: 0.9820 - val_loss: 0.0654
Epoch 5/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 29s 891ms/step - accuracy: 0.9815 - loss: 0.0603 - val_accuracy: 0.9830 - val_loss: 0.0658
Epoch 6/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 29s 910ms/step - accuracy: 0.9790 - loss: 0.0678 - val_accuracy: 0.9830 - val_loss: 0.0588
Epoch 7/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 930ms/step - accuracy: 0.9775 - loss: 0.0462 - val_accuracy: 0.9800 - val_loss: 0.0571
Epoch 8/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 917ms/step - accuracy: 0.9845 - loss: 0.0451 - val_accuracy: 0.984

In [8]:
test_model = keras.models.load_model(
    "feature_extraction_with_data_augmentation.keras"
)
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

/Users/chan/projects/keras-dl/.venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 318 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Users/chan/projects/keras-dl/.venv/lib/python3.11/site-packages/jax/_src/interpreters/mlir.py:1153: UserWarning: Some donated buffers were not usable: ShapedArray(float32[3,3,3,32]), ShapedArray(float32[32]), ShapedArray(float32[32]), ShapedArray(float32[3,3,32,64]), ShapedArray(float32[64]), ShapedArray(float32[64]), ShapedArray(float32[3,3,64,1]), ShapedArray(float32[1,1,64,128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[3,3,128,1]), ShapedArray(float32[1,1,128,128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[1,1,64,128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[3,3,128,1]), ShapedArray(float

32/32 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.9845 - loss: 0.0483
Test accuracy: 0.984


### 9. Prepare for Fine-Tuning

In [9]:
conv_base.trainable = True

for layer in conv_base.layers[:-4]:
    layer.trainable = False
    
for layer in conv_base.layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False

### 10. Train Model (Fine-Tuning)

In [10]:
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    metrics=["accuracy"],
)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning.keras",
        save_best_only=True,
        monitor="val_loss",
    )
]
history = model.fit(
    augmented_train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks,
)

Epoch 1/30


/Users/chan/projects/keras-dl/.venv/lib/python3.11/site-packages/jax/_src/interpreters/mlir.py:1153: UserWarning: Some donated buffers were not usable: ShapedArray(float32[3,3,1536,1]), ShapedArray(float32[1,1,1536,2048]), ShapedArray(float32[2048,256]), ShapedArray(float32[256]), ShapedArray(float32[256,1]), ShapedArray(float32[1]), ShapedArray(float32[3,3,3,32]), ShapedArray(float32[32]), ShapedArray(float32[32]), ShapedArray(float32[32]), ShapedArray(float32[32]), ShapedArray(float32[3,3,32,64]), ShapedArray(float32[64]), ShapedArray(float32[64]), ShapedArray(float32[64]), ShapedArray(float32[64]), ShapedArray(float32[3,3,64,1]), ShapedArray(float32[1,1,64,128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[3,3,128,1]), ShapedArray(float32[1,1,128,128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[1,1,64,128]), ShapedA

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 724ms/step - accuracy: 0.9872 - loss: 0.0301

/Users/chan/projects/keras-dl/.venv/lib/python3.11/site-packages/jax/_src/interpreters/mlir.py:1153: UserWarning: Some donated buffers were not usable: ShapedArray(float32[3,3,1536,1]), ShapedArray(float32[1,1,1536,2048]), ShapedArray(float32[2048,256]), ShapedArray(float32[256]), ShapedArray(float32[256,1]), ShapedArray(float32[1]), ShapedArray(float32[3,3,3,32]), ShapedArray(float32[32]), ShapedArray(float32[32]), ShapedArray(float32[32]), ShapedArray(float32[32]), ShapedArray(float32[3,3,32,64]), ShapedArray(float32[64]), ShapedArray(float32[64]), ShapedArray(float32[64]), ShapedArray(float32[64]), ShapedArray(float32[3,3,64,1]), ShapedArray(float32[1,1,64,128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[3,3,128,1]), ShapedArray(float32[1,1,128,128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[128]), ShapedArray(float32[1,1,64,128]), ShapedA

32/32 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.9870 - loss: 0.0310 - val_accuracy: 0.9820 - val_loss: 0.0672
Epoch 2/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9905 - loss: 0.0264 - val_accuracy: 0.9840 - val_loss: 0.0653
Epoch 3/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.9920 - loss: 0.0196 - val_accuracy: 0.9850 - val_loss: 0.0632
Epoch 4/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 88s 3s/step - accuracy: 0.9930 - loss: 0.0264 - val_accuracy: 0.9840 - val_loss: 0.0619
Epoch 5/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 74s 2s/step - accuracy: 0.9905 - loss: 0.0237 - val_accuracy: 0.9850 - val_loss: 0.0587
Epoch 6/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - accuracy: 0.9900 - loss: 0.0253 - val_accuracy: 0.9860 - val_loss: 0.0569
Epoch 7/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step - accuracy: 0.9890 - loss: 0.0279 - val_accuracy: 0.9850 - val_loss: 0.0581
Epoch 8/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 74s 2s/step - accuracy: 0.9950 - loss: 0.0182 - val_accuracy: 0.9870 - val_loss: 0.0550
Epo

In [13]:
model = keras.models.load_model("fine_tuning.keras")
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 18s 537ms/step - accuracy: 0.9865 - loss: 0.0479
Test accuracy: 0.987
